## How:
    * 214  conda create --name exif
    * 216  conda activate exif
    * 220  conda install pip
    * 221  pip install exif seaborn jupyter
    * 254  pip install reverse_geocoder
    * 265  pip install geopy
    * http://download.geonames.org/export/dump/
      * geonames-all-cities-with-a-population-1000.csv
    
## Where:
    * https://towardsdatascience.com/reverse-geocoding-in-python-a915acf29eb6
    * https://gis.stackexchange.com/questions/357578/python-geopys-nominatim-reverse-geocoding-access-type-e-g-amenity
    * https://gis.stackexchange.com/questions/331144/bulk-reverse-geocoding-with-geopy-using-built-in-rate-limiter
    * https://pypi.org/project/geopy/
    * https://github.com/thampiman/reverse-geocoder
    * https://medium.com/analytics-vidhya/reverse-geocoding-with-geonames-in-python-3b5bb176a26c
    * https://medium.com/spatial-data-science/how-to-extract-gps-coordinates-from-images-in-python-e66e542af354
    * https://stackoverflow.com/questions/72522522/how-to-extract-gps-location-from-heic-files
    
  


In [1]:
import pandas as pd
from datetime import datetime
from PIL import Image
from pillow_heif import register_heif_opener

## Get HEIC metadata for GEOCODING

In [2]:
from PIL import Image
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image.getexif().get_ifd(0x8825)


def get_geotagging(exif):
    geo_tagging_info = {}
    if not exif:
        raise ValueError("No EXIF metadata found")
    else:
        gps_keys = ['GPSVersionID', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude',
                    'GPSAltitudeRef', 'GPSAltitude', 'GPSTimeStamp', 'GPSSatellites', 'GPSStatus', 'GPSMeasureMode',
                    'GPSDOP', 'GPSSpeedRef', 'GPSSpeed', 'GPSTrackRef', 'GPSTrack', 'GPSImgDirectionRef',
                    'GPSImgDirection', 'GPSMapDatum', 'GPSDestLatitudeRef', 'GPSDestLatitude', 'GPSDestLongitudeRef',
                    'GPSDestLongitude', 'GPSDestBearingRef', 'GPSDestBearing', 'GPSDestDistanceRef', 'GPSDestDistance',
                    'GPSProcessingMethod', 'GPSAreaInformation', 'GPSDateStamp', 'GPSDifferential']

        for k, v in exif.items():
            try:
                geo_tagging_info[gps_keys[k]] = str(v)
            except IndexError:
                pass
        return geo_tagging_info

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

register_heif_opener()

In [3]:
my_image = 'sample_images/IMG_1152.HEIC'
#my_image = 'sample_images/IMG_4225.HEIC'
image_info = get_exif(my_image)
results = get_geotagging(image_info)
print(results)

{'GPSLatitudeRef': 'N', 'GPSLatitude': '(37.0, 34.0, 42.91)', 'GPSLongitudeRef': 'E', 'GPSLongitude': '(126.0, 58.0, 38.23)', 'GPSAltitudeRef': "b'\\x00'", 'GPSAltitude': '48.525146484375', 'GPSTimeStamp': '(7.0, 15.0, 0.0)', 'GPSSpeedRef': 'K', 'GPSSpeed': '0.0', 'GPSImgDirectionRef': 'T', 'GPSImgDirection': '273.68064876957493', 'GPSDestBearingRef': 'T', 'GPSDestBearing': '273.68064876957493', 'GPSDateStamp': '2019:06:19'}


In [4]:
dateValue = datetime.strptime(results['GPSDateStamp'], "%Y:%m:%d").date()
year, month, day = map(int, results['GPSDateStamp'].strip('()').split(':'))

degrees, minutes, seconds = map(float, results['GPSLatitude'].strip('()').split(','))
direction = results['GPSLatitudeRef']
dd_lat = dms2dd(degrees, minutes, seconds, direction)

degrees, minutes, seconds = map(float, results['GPSLongitude'].strip('()').split(','))
direction = results['GPSLongitudeRef']
dd_long = dms2dd(degrees, minutes, seconds, direction)

print (year, month, day, dd_lat, dd_long)

2019 6 19 37.578586111111115 126.97728611111111


## Reverse GeoCode

In [5]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [6]:
geolocator = Nominatim(user_agent="application")

reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

location = reverse((dd_lat, dd_long), language='en', exactly_one=True)

print (location.raw)

{'place_id': 299911227, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 5501517, 'lat': '37.57975395', 'lon': '126.97668069886859', 'display_name': 'Gyeongbokgung Palace, Hyoja-ro, Jeokseon-dong, Cheongunhyoja-dong, Jongno-gu, Seoul, 03044, South Korea', 'address': {'tourism': 'Gyeongbokgung Palace', 'road': 'Hyoja-ro', 'quarter': 'Jeokseon-dong', 'suburb': 'Cheongunhyoja-dong', 'borough': 'Jongno-gu', 'city': 'Seoul', 'ISO3166-2-lvl4': 'KR-11', 'postcode': '03044', 'country': 'South Korea', 'country_code': 'kr'}, 'boundingbox': ['37.5758581', '37.583667', '126.973755', '126.9800491']}


In [7]:
print (location.raw['display_name'], 
       location.raw['address']['city'],
       location.raw['address']['country'],
      )

Gyeongbokgung Palace, Hyoja-ro, Jeokseon-dong, Cheongunhyoja-dong, Jongno-gu, Seoul, 03044, South Korea Seoul South Korea


## Static File Lookup for City State

In [8]:
import reverse_geocoder as rg
import io

In [9]:
coordinates = (51.5214588,-0.1729636),(9.936033, 76.259952),(37.38605,-122.08385)

results = rg.search(coordinates) # default mode = 2

print (results)

Loading formatted geocoded file...
[{'lat': '51.51116', 'lon': '-0.18426', 'name': 'Bayswater', 'admin1': 'England', 'admin2': 'Greater London', 'cc': 'GB'}, {'lat': '9.93988', 'lon': '76.26022', 'name': 'Cochin', 'admin1': 'Kerala', 'admin2': 'Ernakulam', 'cc': 'IN'}, {'lat': '37.38605', 'lon': '-122.08385', 'name': 'Mountain View', 'admin1': 'California', 'admin2': 'Santa Clara County', 'cc': 'US'}]


In [10]:
geo = rg.RGeocoder(mode=2, verbose=True, stream=io.StringIO(open('geonames-all-cities-with-a-population-1000.csv', encoding='utf-8').read()))
coordinates = (dd_lat, dd_long),(9.936033, 76.259952),(37.38605,-122.08385)
results = geo.query(coordinates)
print (results)

[{'lat': '37.566', 'lon': '126.9784', 'name': 'Seoul', 'admin1': 'Seoul', 'admin2': '', 'cc': 'KR'}, {'lat': '9.93988', 'lon': '76.26022', 'name': 'Cochin', 'admin1': 'Kerala', 'admin2': 'Ernakulam', 'cc': 'IN'}, {'lat': '37.38605', 'lon': '-122.08385', 'name': 'Mountain View', 'admin1': 'California', 'admin2': 'Santa Clara County', 'cc': 'US'}]


In [11]:
for c in results:
    l = [c['lat'], c['lon'], c['name'], c['admin1'], c['admin2']]
    print (','.join(l))

37.566,126.9784,Seoul,Seoul,
9.93988,76.26022,Cochin,Kerala,Ernakulam
37.38605,-122.08385,Mountain View,California,Santa Clara County


## JPG

In [12]:
from exif import Image
def image_coordinates(image_path):
    with open(image_path, 'rb') as src:
        img = Image(src)
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (decimal_coords(img.gps_latitude,
                      img.gps_latitude_ref),
                      decimal_coords(img.gps_longitude,
                      img.gps_longitude_ref))
        except AttributeError:
            print ('No Coordinates')
    else:
        print ('The Image has no EXIF information')
        print(f"Image {src.name}, OS Version:{img.get('software', 'Not Known')} ------")
        print(f"Was taken: {img.datetime_original}, and has coordinates:{coords}")

# Done - ZZZZZ